In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from pipelines.preprocessors import *
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Cargar los datasets completos
df1 = pd.read_parquet('../../data/raw/sample_data_0006_part_00.parquet')
df2 = pd.read_parquet('../../data/raw/sample_data_0007_part_00.parquet')

# Tomar una fracción de cada dataset
FRAC = 0.20
SEED = 42
sample_df1 = df1.sample(frac=FRAC, random_state=SEED)
sample_df2 = df2.sample(frac=FRAC, random_state=SEED)

# Concatener los datasets
data = pd.concat([sample_df1, sample_df2], ignore_index=True)

In [3]:
# Tipos de datos
data.dtypes

merchant_id                   object
_id                           object
subsidiary                    object
transaction_date      datetime64[ns]
account_number                object
user_id                       object
transaction_amount            object
transaction_type              object
dtype: object

In [4]:
# Ver cuántos registros están duplicados
print(f"Número de registros originalmente: {len(data)}")

# Identificar duplicados en la columna user_id
duplicated_rows= data[data.duplicated(keep=False)]

# Ver cuántos registros están duplicados
print(f"Número de registros duplicados: {len(duplicated_rows)}")

Número de registros originalmente: 4303384
Número de registros duplicados: 0


In [5]:
# Suponiendo que 'data' es tu DataFrame y 'transaction_date' es una columna de tipo datetime
fecha_minima = data['transaction_date'].min()
fecha_maxima = data['transaction_date'].max()

print(f'Fecha mínima: {fecha_minima}')
print(f'Fecha máxima: {fecha_maxima}')

Fecha mínima: 2021-01-01 00:00:40
Fecha máxima: 2021-11-30 23:57:57


In [6]:
# Verificar si cada account_number está asociado a un único user_id
unique_accounts = data.groupby('user_id')['account_number'].nunique()

# Analizar resultados
if unique_accounts.max() == 1:
    print('La relación es 1:1 (una cuenta por usuario).')
else:
    print('La relación no es 1:1. Hay cuentas compartidas por varios usuarios.')

La relación no es 1:1. Hay cuentas compartidas por varios usuarios.


In [7]:
# 1) Ordenar el DataFrame
data = data.sort_values(by=['user_id', 'transaction_date'])

def check_fraccionamiento_24h(user_df: pd.DataFrame, min_count: int=2):
    """
    Función que evalúa, para cada transacción de un usuario, cuántas transacciones
    ocurren en la ventana de [t - 24h, t]. Si el conteo de transacciones en esa
    ventana es >= min_count, se considera 'fraccionada'.

    Parámetros:
    -----------
    user_df: pd.DataFrame
        Subconjunto de datos correspondientes a un usuario específico (user_id).
    min_count: int
        Número mínimo de transacciones que activan la bandera de fraccionamiento.
        Por defecto es 2, pero puede ajustarse según necesidades de negocio.

    Retorna:
    --------
    flags: list of bool
        Lista booleana del mismo tamaño que user_df indicando True (fraccionada)
        o False (no fraccionada) para cada transacción.
    """
    times = user_df['transaction_date'].values
    flags = list()
    for i in range(len(times)):
        current_time = times[i]
        window_mask = (
            (times >= current_time - np.timedelta64(24, 'h')) & 
            (times <= current_time)
        )
        window_sub = user_df[window_mask]
        count_tx = len(window_sub)
        
        if count_tx >= min_count:
            flags.append(True)
        else:
            flags.append(False)
            
    return flags

# 2) Generar el array de valores booleanos sin asignarlos al DataFrame todavía
bool_flags = (
    data
    .groupby('user_id', group_keys=False)
    .apply(lambda df: check_fraccionamiento_24h(df, min_count=2))
    .explode()        # Separa la lista booleana por filas
    .astype(bool)     # Asegurarnos de que sea tipo bool
    .values           # Convertir a array de NumPy
)

# 3) Crear directamente la columna de texto 'fraction_flag'
data['fraction_flag'] = np.where(
    bool_flags,  # type: ignore
    'FRACCIONADA', 
    'NO_FRACCIONADA'
)

In [8]:
# Revisar resultados
cols = ['user_id', 'transaction_date', 'transaction_amount', 'transaction_type', 'fraction_flag']
data[data['fraction_flag'] == 'FRACCIONADA'].sample(10, random_state=SEED)[cols]

,user_id,transaction_date,transaction_amount,transaction_type,fraction_flag
2970481,d3cee2664b169027f100b4bcb0c7edd9,2021-09-18 18:21:38,59.44455012,DEBITO,FRACCIONADA
2438663,db4f8b7e6634b4f758da95e063afd02c,2021-09-19 13:58:39,83.22237017,DEBITO,FRACCIONADA
2976635,b4ca5e0f8697bad5c0baa8f967eb3324,2021-06-10 22:43:52,11.88891002,DEBITO,FRACCIONADA
939296,304e1d139f265fa3c711b0dce9d8ec71,2021-11-14 08:35:19,237.77820049,CREDITO,FRACCIONADA
4094428,a72a7f3463d9e2f0d21129fe9aff81ad,2021-02-13 17:30:30,5.94445501,CREDITO,FRACCIONADA
2097824,b10fad86495359fe0fa1927af794db10,2021-10-05 13:06:29,475.55640098,CREDITO,FRACCIONADA
855384,fc48dd3fafeefecdbe7c5b0874f29dd5,2021-01-15 19:04:11,5.94564390,DEBITO,FRACCIONADA
1531865,0a638d39a994a4e555447c951f4a00ec,2021-01-30 10:14:55,5.94802168,CREDITO,FRACCIONADA
1689105,d23a1b6e9cc006df05d01a87aca25ce9,2021-02-13 17:26:59,6.53890051,DEBITO,FRACCIONADA
2686780,23bb152745a64195d264364d3620329a,2021-02-10 15:28:41,214.00038044,DEBITO,FRACCIONADA


In [9]:
# Revisar resultados
data[data['user_id'] == '985f7119be1883130b85729cb6c30607'][cols]

,user_id,transaction_date,transaction_amount,transaction_type,fraction_flag
91901,985f7119be1883130b85729cb6c30607,2021-01-09 05:52:49,5.94445501,DEBITO,NO_FRACCIONADA
1518983,985f7119be1883130b85729cb6c30607,2021-01-09 05:57:49,5.94445501,DEBITO,FRACCIONADA
664137,985f7119be1883130b85729cb6c30607,2021-01-09 06:33:10,5.94445501,DEBITO,FRACCIONADA
1628651,985f7119be1883130b85729cb6c30607,2021-01-09 07:51:23,5.94683279,DEBITO,FRACCIONADA
779268,985f7119be1883130b85729cb6c30607,2021-01-09 08:57:55,5.94445501,DEBITO,FRACCIONADA
...,...,...,...,...,...
4216698,985f7119be1883130b85729cb6c30607,2021-02-13 12:12:53,6.06334411,CREDITO,FRACCIONADA
2949444,985f7119be1883130b85729cb6c30607,2021-02-13 12:22:57,6.30112231,CREDITO,FRACCIONADA
4070893,985f7119be1883130b85729cb6c30607,2021-02-13 15:16:26,6.18223321,CREDITO,FRACCIONADA
4054859,985f7119be1883130b85729cb6c30607,2021-02-13 15:54:29,6.18223321,CREDITO,FRACCIONADA


In [10]:
# Revisar resultados
data[data['user_id'] == '3c3ba58ade14ea490ec84317548a9455'][cols]

,user_id,transaction_date,transaction_amount,transaction_type,fraction_flag
1438686,3c3ba58ade14ea490ec84317548a9455,2021-01-11 08:58:09,5.94683279,DEBITO,NO_FRACCIONADA
651471,3c3ba58ade14ea490ec84317548a9455,2021-01-11 14:48:44,5.94683279,DEBITO,FRACCIONADA
1298277,3c3ba58ade14ea490ec84317548a9455,2021-01-12 08:44:02,5.94683279,DEBITO,FRACCIONADA
89707,3c3ba58ade14ea490ec84317548a9455,2021-01-12 08:48:03,5.94683279,DEBITO,FRACCIONADA
377560,3c3ba58ade14ea490ec84317548a9455,2021-01-12 08:52:11,5.94683279,DEBITO,FRACCIONADA
...,...,...,...,...,...
356993,3c3ba58ade14ea490ec84317548a9455,2021-02-16 08:30:07,5.94445501,CREDITO,FRACCIONADA
1555739,3c3ba58ade14ea490ec84317548a9455,2021-02-16 14:24:44,5.94445501,CREDITO,FRACCIONADA
1348995,3c3ba58ade14ea490ec84317548a9455,2021-02-16 14:42:12,5.94445501,CREDITO,FRACCIONADA
1885334,3c3ba58ade14ea490ec84317548a9455,2021-02-16 14:44:25,5.94445501,CREDITO,FRACCIONADA


In [11]:
# Solo el 7.7% del total de las transacciones son fraccionadas
data['fraction_flag'].value_counts(normalize=True)

fraction_flag
NO_FRACCIONADA    0.922137
FRACCIONADA       0.077863
Name: proportion, dtype: float64

In [12]:
# No hay montos negativos
data[data['transaction_amount'] < 0]

,merchant_id,_id,subsidiary,transaction_date,account_number,user_id,transaction_amount,transaction_type,fraction_flag


In [13]:
# Preprocesadores
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('numeric_columns', NumericColumnTransformer()),
    ('duplicaed_columns', DropDuplicateColumnsTransformer()),
    ('duplicated_rows', DropDuplicatedRowsTransformer()),
    ('date_columns', DateColumnsTransformer()),
    ('columns_rename', ColumnsRenameTransformer(lambda col: str(col).lower().strip().replace(' ', '_'))),
    ('nan_values', FillMissingValuesTransformer()),
])

# Aplicar el pipeline a los datos
data_processed = pd.DataFrame(pipe.fit_transform(data))

In [14]:
# Tipos de datos correctos
data_processed.dtypes

merchant_id                   object
_id                           object
subsidiary                    object
transaction_date      datetime64[ns]
account_number                object
user_id                       object
transaction_amount           float64
transaction_type              object
fraction_flag                 object
dtype: object

In [15]:
# Valores faltantes
data_processed.isnull().sum()

merchant_id           0
_id                   0
subsidiary            0
transaction_date      0
account_number        0
user_id               0
transaction_amount    0
transaction_type      0
fraction_flag         0
dtype: int64

In [16]:
# Exportar
data_processed.to_parquet('../../data/processed/data_processed.parquet', index=False)